You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

In [522]:
import pandas as pd 
import numpy as np

In [524]:
times = pd.read_csv("C:/Users/maurizio/Desktop/Maurizio/progetto cs/timesData.csv", na_values = "-", thousands = ",")                                                                                                
shanghai = pd.read_csv('C:/Users/maurizio/Desktop/Maurizio/progetto cs/shanghaiData.csv')
cwur = pd.read_csv('C:/Users/maurizio/Desktop/Maurizio/progetto cs/cwurData.csv')

### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

In [525]:
times_most_recent = times.iloc[times.groupby("university_name")["year"].idxmax()]
times_last_recent = times.iloc[times.groupby("university_name")["year"].idxmin()]

### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

In [526]:
improvement = pd.merge(times_most_recent, times_last_recent, on = "university_name", suffixes = ["_most_recent",
                                                                                                 "_last_recent"])

In [527]:
improvement["difference_in_income"] = improvement["income_most_recent"] - improvement["income_last_recent"]

### 3. Find the university with the largest increase computed in the previous point

In [529]:
improvement.iloc[improvement["difference_in_income"].idxmax()]

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

In [530]:
shanghai_most_recent = shanghai.iloc[shanghai.groupby('university_name')['year'].idxmax()]
cwur_most_recent = cwur.iloc[cwur.groupby('institution')['year'].idxmax()]

In [531]:
all_rankings = pd.merge(pd.merge(shanghai_most_recent, 
                        times_most_recent, on = "university_name", suffixes = ["_shanghai", "_times"]),
                        cwur_most_recent, left_on = "university_name", right_on = "institution")

In [532]:
all_rankings.rename(columns = {"world_rank": "world_rank_cwur" }, inplace = True)

In [533]:
all_rankings["world_rank_shang_min"] = all_rankings["world_rank_shanghai"].str.extract("(\d+)-*").astype(float)
all_rankings["world_rank_shang_max"] = all_rankings["world_rank_shanghai"].str.extract("-(\d+)").astype(float)

In [534]:
all_rankings["world_rank_times_min"] = all_rankings["world_rank_times"].str.extract("=*(\d+)-*").astype(float)
all_rankings["world_rank_times_max"] = all_rankings["world_rank_times"].str.extract("-(\d+)").astype(float)

In [535]:
def calcola_media_times(df):
    if pd.isna(df["world_rank_times_max"]):
        return df["world_rank_times_min"]
    else: 
        return (df["world_rank_times_min"]+df["world_rank_times_max"])/2

In [536]:
def calcola_media_shanghai(df):
    if pd.isna(df["world_rank_shang_max"]):
        return df["world_rank_shang_min"]
    else:
        return (df["world_rank_shang_min"]+df["world_rank_shang_max"])/2

In [537]:
all_rankings["world_rank_times_mean"] = all_rankings[["world_rank_times_min", 
                                                      "world_rank_times_max"]].apply(calcola_media_times, axis = 1)

In [538]:
all_rankings["world_rank_shang_mean"] = all_rankings[["world_rank_shang_min",
                                                      "world_rank_shang_max"]].apply(calcola_media_shanghai, axis =1)

In [539]:
all_rankings["diff_shang_time"] = (all_rankings["world_rank_shang_mean"] - all_rankings["world_rank_times_mean"]).abs()
all_rankings["diff_shang_cwur"] = (all_rankings["world_rank_shang_mean"] - all_rankings["world_rank_cwur"]).abs()
all_rankings["diff_times_cwur"] = (all_rankings["world_rank_times_mean"] - all_rankings["world_rank_cwur"]).abs()

In [540]:
trova_differenza_massima = all_rankings[["university_name","diff_shang_time", "diff_shang_cwur", "diff_times_cwur"]]

In [482]:
trova_differenza_massima.set_index("university_name", inplace = True)

In [517]:
trova_differenza_massima.max(axis = 1)

### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

In [541]:
pd.options.mode.chained_assignment = None #to avoid useless settingwithcopy warning

In [542]:
times_most_recent["female_percentage"] = times_most_recent["female_male_ratio"].str.extract("(\d+) :").astype(float) 
times_most_recent["num_stud"] = times_most_recent["num_students"].astype(float)

In [543]:
times_most_recent["number_of_female"] = times_most_recent["num_students"]*(times_most_recent["female_percentage"])

### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [544]:
times_most_recent["male_perc"] = times_most_recent["female_male_ratio"].str.extract(" : (\d+)").astype(float)

In [545]:
def calcola_ratio(df):
    if df["male_perc"] == 0:
        return 1
    else:
        return df["female_percentage"]/df["male_perc"]

In [546]:
times_most_recent["ratio"] = times_most_recent[["female_percentage", "male_perc"]].apply(calcola_ratio, axis = 1 )
average_ratio = times_most_recent["ratio"].mean()

In [547]:
#solution:
universities_below_avg_ratio = times_most_recent[times_most_recent["ratio"] < average_ratio]["university_name"]

### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [548]:
below_average = times_most_recent[times_most_recent["ratio"] < average_ratio].groupby("country")["num_stud"].sum()

In [549]:
number_of_students_in_each_country = times_most_recent.groupby("country")["num_stud"].sum()


In [550]:
solution_point_seven = below_average/number_of_students_in_each_country

### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row with missing `country_name` or `series_name`

In [554]:
attainment = pd.read_csv("C:/Users/maurizio/Desktop/Maurizio/progetto cs/educational_attainment_supplementary_data.csv")

In [555]:
attainment.dropna(how = "any", subset = ["country_name", "series_name"], inplace = True)

### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

In [556]:
modified_attainment = pd.melt(attainment, value_vars= ['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'], id_vars = ["country_name", "series_name"])

### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [557]:
appearence_of_university_in_rankings = dict()
times_university = set(times["university_name"])
shanghai_university = set(shanghai["university_name"])
cwur_univeristy = set(cwur["institution"])

In [558]:
for university in times_university:
    appearence_of_university_in_rankings[university] = 1
for university in shanghai_university:
    if university not in appearence_of_university_in_rankings:
        appearence_of_university_in_rankings[university] = 1
    else:
        appearence_of_university_in_rankings[university] += 1
for university in cwur_univeristy:
    if university not in appearence_of_university_in_rankings:
        appearence_of_university_in_rankings[university] = 1
    else:
        appearence_of_university_in_rankings[university] += 1 

### 11. In the times ranking, compute the number of times each university appears

In [559]:
point_eleven = times.groupby("university_name")["university_name"].count()

### 12. Find the universities that appear at most twice in the times ranking.

In [560]:
point_twelve = point_eleven[point_eleven <= 2]

### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [561]:
times["intermedio"] = times["world_rank"].str.extract("(\d+)").astype(float) 
times["intermedio_2"] = times["world_rank"].str.extract("-(\d+)").astype(float)
shanghai["intermedio"] = shanghai["world_rank"].str.extract("(\d+)").astype(float)
shanghai["intermedio_2"] = shanghai["world_rank"].str.extract("-(\d+)").astype(float)

In [562]:
def calcola_media(df):
    if pd.isna(df["intermedio_2"]):
        return df["intermedio"]
    else:
        return (df["intermedio"]+df["intermedio_2"])/2

In [563]:
times["rank_adj_times"] = times.apply(calcola_media, axis = 1)
shanghai["rank_adj_shang"] = shanghai.apply(calcola_media, axis = 1)

In [564]:
times_2012 = times[times["year"] == 2012][["rank_adj_times", "university_name"]]
times_2013 = times[times["year"] == 2013][["rank_adj_times", "university_name"]]
times_2014 = times[times["year"] == 2014][["rank_adj_times", "university_name"]]
times_2015 = times[times["year"] == 2015][["rank_adj_times", "university_name"]]

In [565]:
shanghai_2012 = shanghai[shanghai["year"] == 2012][["rank_adj_shang", "university_name"]]
shanghai_2013 = shanghai[shanghai["year"] == 2013][["rank_adj_shang", "university_name"]]
shanghai_2014 = shanghai[shanghai["year"] == 2014][["rank_adj_shang", "university_name"]]
shanghai_2015 = shanghai[shanghai["year"] == 2015][["rank_adj_shang", "university_name"]]

In [566]:
cwur_2012 = cwur[cwur["year"] == 2012][["world_rank", "institution"]]
cwur_2013 = cwur[cwur["year"] == 2013][["world_rank", "institution"]]
cwur_2014 = cwur[cwur["year"] == 2014][["world_rank", "institution"]]
cwur_2015 = cwur[cwur["year"] == 2015][["world_rank", "institution"]]

In [567]:
ranking_2012 = pd.merge(pd.merge(times_2012, shanghai_2012, on = "university_name", suffixes = ("_times", "_shang")), 
                                                   cwur_2012, left_on = "university_name", right_on = "institution")
ranking_2013 = pd.merge(pd.merge(times_2013, shanghai_2013, on = "university_name", suffixes = ("_times", "_shang")), 
                             cwur_2013, left_on = "university_name", right_on = "institution")
ranking_2014 = pd.merge(pd.merge(times_2014, shanghai_2014, on = "university_name", suffixes = ("_times", "_shang")), 
                             cwur_2014, left_on = "university_name", right_on = "institution")
ranking_2015 = pd.merge(pd.merge(times_2014, shanghai_2015, on = "university_name", suffixes = ("_times", "_shang")), 
                             cwur_2015, left_on = "university_name", right_on = "institution")

In [568]:
ranking_2012["stessa_posizione"] = np.where((ranking_2012["rank_adj_times"] == ranking_2012["rank_adj_shang"])
                                 & (ranking_2012["rank_adj_times"] == ranking_2012["world_rank"]), "uguali", "diversi")

In [569]:
ranking_2013["stessa_posizione"] = np.where((ranking_2013["rank_adj_times"] == ranking_2013["rank_adj_shang"])
                                 & (ranking_2013["rank_adj_times"] == ranking_2013["world_rank"]), "uguali", "diversi")

In [570]:
ranking_2014["stessa_posizione"] = np.where((ranking_2014["rank_adj_times"] == ranking_2014["rank_adj_shang"])  
                                 & (ranking_2014["rank_adj_times"] == ranking_2014["world_rank"]), "uguali", "diversi")

In [511]:
ranking_2015["stessa_posizione"] = np.where((ranking_2015["rank_adj_times"] == ranking_2015["rank_adj_shang"])  
                                 & (ranking_2015["rank_adj_times"] == ranking_2015["world_rank"]), "uguali", "diversi")

In [518]:
ranking_2013[ranking_2013["stessa_posizione"] ==  "uguali"]

In [519]:
ranking_2013[ranking_2013["stessa_posizione"] ==  "uguali"]

In [520]:
ranking_2014[ranking_2014["stessa_posizione"] ==  "uguali"]

In [521]:
ranking_2015[ranking_2015["stessa_posizione"] ==  "uguali"]